**Loan Approval Prediction**

1. Essential Imports and Global Settings

In [1]:
# ## 1. Essential Imports and Global Settings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Setup
warnings.filterwarnings("ignore")
sns.set(style="whitegrid")

2. Configuration and Dataset Path

In [2]:
# ## 2. Configuration and Dataset Path
DATA_PATH = "../data/loan_data.csv"
SEED = 42